In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.spatial import distance
from sklearn.linear_model import LinearRegression
import sklearn 
import scipy
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors

In [2]:
os.chdir("/content/drive/MyDrive/Winter_Research")

In [3]:
KS = (32, 6279, 11, 3) 
MT = (29, 6554, 11, 3) 
TX = (34, 6524, 11, 3) 
OH =(19, 5712, 11, 3)
CA = (43, 6569, 11, 3) 

In [4]:
# New Cloud Data
# CA -- 43 images, 72264 pixels, save shape (43, 6569, 11, 3)
# KS -- 32 images, 69071 pixels, save shape (32, 6279, 11, 3)
# MT -- 23 images, 72099 pixels, save shape (23, 6554, 11, 3)
# TX -- 32 images, 71764 pixels, save shape (32, 6524, 11, 3)
# OH -- 12 images, 62827 pixels, save shape (12, 5711, 11, 3)

In [5]:
loadedArr = np.loadtxt("Rois/Roi_2/Cloud_Mask_v2.csv", delimiter=',')
reshape = loadedArr.reshape(32, -1, 5)

In [6]:
reshape.shape

(32, 69071, 5)

In [ ]:
imgs = []

In [ ]:
pix_width = 11
plot = False
name = ""
save = False
for i in tqdm(range(reshape.shape[0])): 
  print(i) 
  first_pic = reshape[i]
  row_num = first_pic.shape[0] // pix_width
  first_pic = first_pic[:pix_width*row_num]
  lat = first_pic[:, 4]
  lon = first_pic[:, 3]
  coords = np.hstack([lon[:, np.newaxis], lat[:, np.newaxis]])
  x_cent, y_cent, fit = fit_and_splice(lon, lat)
  if plot:
    print(lat.shape)
    new_x = np.array(lon).reshape(-1,1)
    new_y = np.array(lat)
    plt.scatter(lon, lat)
    plt.xlabel("Longitude Coord")
    plt.ylabel("LatitudeCoord")
    #plt.scatter(new_x, new_y)
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = fit.intercept_ + fit.coef_ * x_vals
    plt.plot(x_vals, y_vals, '--')
    plt.savefig("Road_Fit.png")
    plt.show()
    if save:
      plt.savefig(name)
    break
  new_img = splice(first_pic, coords, x_cent, y_cent, pix_width)
  new_sorted_img = sort_arr(new_img, fit)
  #new_img_rgb = (255*((new_sorted_img - 100)/3500)).astype('uint8')
  # plt.scatter(lat, lon)
  # plt.show()
  imgs.append(new_sorted_img)


  0%|          | 0/32 [00:00<?, ?it/s]

0


  3%|▎         | 1/32 [02:58<1:32:13, 178.50s/it]

1


  6%|▋         | 2/32 [05:59<1:29:38, 179.28s/it]

...

 94%|█████████▍| 30/32 [1:28:11<05:48, 174.31s/it]

30


 97%|█████████▋| 31/32 [1:31:06<02:54, 174.47s/it]

31


100%|██████████| 32/32 [1:34:01<00:00, 176.29s/it]


In [ ]:
len(imgs)

32

In [ ]:
new_np = np.array(imgs)

In [ ]:
new_np.shape

(32, 6279, 11, 3)

In [ ]:
arrReshaped = new_np.reshape(new_np.shape[0]*new_np.shape[1], -1)
np.savetxt("Rois/Roi_2/Straight_images_v2.csv", arrReshaped, delimiter=',')

In [ ]:
np.unique(new_np, axis=0).shape

(43, 6569, 11, 3)

In [ ]:
arr = np.loadtxt("Rois/Roi_1/Straight_images_v2.csv", delimiter=',')
compress_back = arr.reshape((43, 6569, 11, 3)) 

In [ ]:
compress_3000 = compress_back / 3000

In [ ]:
broken = {"TX" : "MT" : [8, 9, 10], "KS" : [8, 16, 18, 19, 26, 31]}

In [ ]:
from PIL import Image
import numpy as np

# data = np.random.random((100,100))

# #Rescale to 0-255 and convert to uint8
# rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)

# im = Image.fromarray(rescaled)
# im.save('test.png')

In [ ]:
rescaled = (255.0 / compress_back.max() * (compress_back - compress_back.min())).astype(np.uint8)

In [ ]:
os.mkdir("CA_straight")

In [ ]:
#new_img_rgb = (255*((new_sorted_img - 100)/3500)).astype('uint8')
for i in range(compress_3000.shape[0]):
  print(i)  
  new_img = compress_3000[i]
  print(new_img.shape)
  print(np.max(np.mean(new_img, axis=2)))
  disp = new_img[1000:1050]
  rescaled = (255*disp).astype(np.uint8)
  plt.imshow(rescaled)
  
  im = Image.fromarray(rescaled)
  im.save('CA_straight/cali_img' + str(i) + '.png')
  plt.show()

In [ ]:
new_img_rgb

In [ ]:
def fit_and_splice(lon, lat):  
  new_x = np.array(lon).reshape(-1,1)
  new_y = np.array(lat)
  fit = LinearRegression().fit(new_x, new_y)
  score = fit.score(new_x, new_y)
  # plt.xlabel("Longitude Coord")
  # plt.ylabel("LatitudeCoord")
  # plt.scatter(new_x, new_y)
  # axes = plt.gca()
  # x_vals = np.array(axes.get_xlim())
  # y_vals = fit.intercept_ + fit.coef_ * x_vals
  predictions = fit.predict(new_x)
  splices = [x / row_num for x in range(row_num + 1)]
  buckets = scipy.stats.mstats.mquantiles(predictions, prob=splices)
  y_cent = [(buckets[i] + buckets[i + 1]) / 2 for i in range(len(buckets) - 1)]
  x_cent = (y_cent - fit.intercept_) / fit.coef_
  return x_cent, y_cent, fit
  # print(score)
  # plt.plot(x_vals, y_vals, '--')
  # plt.savefig("Road_Fit.png")
  # plt.show()

In [ ]:
def splice(pic, coords, x_cent, y_cent, pix_width):  
  new_img = np.zeros((x_cent.shape[0], pix_width , 5))
  centers = zip(x_cent, y_cent)
  temp_coords = coords.copy()
  for i in range(pix_width):
    x_plot = []
    y_plot = []  
    for j, (x, y) in enumerate(zip(x_cent, y_cent)):
      cent = np.array((x, y))
      dist = np.linalg.norm(temp_coords - cent, axis=1)
      index = dist.argmin()
      new_img[j][i][:] = pic[index][:]
      temp_coords[index] = [0, 0]
      if temp_coords[index][0] != 0 and temp_coords[index][1] != 0:
        print(index, temp_coords[index])
  return new_img

In [ ]:
def sort_arr(new_img, fit):  
  new_sorted_img = np.zeros((new_img.shape[0], new_img.shape[1], 3))
  score = 0
  scores = []
  for i in range(new_img.shape[0]):
    row = new_img[i]
    new_row = np.zeros((pix_width, 3))
    center = np.array([x_cent[i], y_cent[i]])
    # print(center)
    # break
    lat_lon = np.hstack([row[:,3][:, np.newaxis], row[:,4][:, np.newaxis]]) 
    dist = np.linalg.norm(lat_lon - center, axis=1)
    score += np.sum(dist)
    scores.append(np.sum(dist))
    preds = np.sign(row[:, 3] - fit.coef_ * row[:, 4] + fit.intercept_)
    #print(dist.shape, preds.shape)
    rankings = np.multiply(dist, preds)
    indicies = rankings.argsort()
    new_sorted_img[i] = row[:, 0:3][indicies]
  return new_sorted_img